In [15]:
#the rating of a movie by looking at the 10 movies closest to it in genre and popularity
#how about the closest 15 movies?

#step:#import columns and create a dframe
      #define a function to cal the distance between 2 movies
      #define a function to cal the KNN of your movies
        
        
import pandas as pd
import numpy as np
from scipy import spatial

cols=['user_id','movie_id','ratings']
rating_data = pd.read_csv(r"D:\Dig\ml-100k\u.data",sep="\t",names=cols,usecols=range(3)) #usecols -> num of cols we want
rating_data.head()

,user_id,movie_id,ratings
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [16]:
#now we will group all info by movie_id and then we will try to compute total number of ratings
#And once we have that we will cal avg num of rating for each movie

#For that we will apply group by method on our rating dataframe and then we will aggregate the result by rating and mean

movie_properties=rating_data.groupby('movie_id').agg({'ratings':[np.size,np.mean]})
movie_properties.head()

ratings          
            size      mean
movie_id                  
1            452  3.878319
2            131  3.206107
3             90  3.033333
4            209  3.550239
5             86  3.302326

In [17]:
#for movie 1 , 452 ppl's ,mean of rating is 3.878
#for movie 2 , 131 ppl's ,mean of rating is 3.206
#and so on...

#now we will normalize the mean_rating ....example : 3.87 may be fall around 0.75


# For machine learning, every dataset does not require normalization. It is required only when features have different ranges.
movie_num_raings=pd.DataFrame(movie_properties['ratings']['size'])

#Formula for normalization
#avg = df['X'].mean()
#diff = df['X'].max() - df['X'].min()
#new_df = df['X'].apply(lambda x: (x-avg)/diff) 

movie_normalized_num_raing = movie_num_raings.apply(lambda x: (x - np.min(x))/(np.max(x)-np.min(x)))
#Here we r finding the max & min num of ratings in ur dataset & then we find the range & normalize everything against that range
movie_normalized_num_raing.head()

,size
movie_id,
1,0.774914
2,0.223368
3,0.152921
4,0.357388
5,0.146048


In [18]:
#u.times have 19 different type of genras it has value of 0 or 1 where 1 means that movies is matching that genra & 0 means not
#We will create a dict
#This dict will contain it name,genra_value & we will have a normalized score for its popularity & the avg rating for that movie

movie_dict = {}
with open(r'D:\Dig\ml-100k\u.item') as f:
    temp=''

#Once we open the file we will iterate through every line of the file i.e 1 line at a time
#And the we will strip out new line at the end
#Then we will extract movie_id, movie_name & its genra
#Once we have that we will create dict which maps movie_ids throught there name , genres & also there normalized rating

    for line in f:
        fields = line.rstrip('\n').split('|')
        movie_id=int(fields[0])
        name=fields[1]
        genres=fields[5:25]
        genres=list(map(int,genres))
        
        movie_dict[movie_id]=(name,genres,movie_normalized_num_raing.loc[movie_id].get('size'),
                             movie_properties.loc[movie_id].ratings)

In [19]:
movie_dict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 0.7749140893470791,
 size    452.000000
 mean      3.878319
 Name: 1, dtype: float64)

In [20]:
#lets create a function which cal the distance between 2 movies based on there popularity and ratings and genras

def compute_distance(a,b):
    genres_a=a[1]
    genres_b=b[1]
    
    #cal distance between this 2 movies
    g_distance = spatial.distance.cosine(genres_a,genres_b)
    
    popularity_a=a[2]
    popularity_b=b[2]
    
    #abs give distance from zero ,abs is never negative
    popularity_dis=abs(popularity_a-popularity_b)
    
    return g_distance + popularity_dis

compute_distance(movie_dict[2],movie_dict[3])

0.49309646620556324

In [21]:
print (movie_dict[2])
print (movie_dict[3])

('GoldenEye (1995)', [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 0.22336769759450173, size    131.000000
mean      3.206107
Name: 2, dtype: float64)
('Four Rooms (1995)', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 0.15292096219931273, size    90.000000
mean     3.033333
Name: 3, dtype: float64)


In [22]:
#Define a function to cal the KNN of ur movie
from sklearn.neighbors import KNeighborsClassifier
import operator
def get_neighbors(movie_id,k):
    distances = []
    
    #compute distance between movie num 1 and all other movies
    #Once we have that we will sort the results based on there distance score
    for movie in movie_dict:
        #For movie in movie_dict we will iteratet through each movie that we have and if its a diff movie than what we are
        #looking for it will compute the distance from previous result 
        if(movie != movie_id):
            dist = compute_distance(movie_dict[movie_id],movie_dict[movie])
            distances.append((movie,dist))
    distances.sort(key = operator.itemgetter(1))
    
    #Now we will got to each k value from 1:10 then we will append distaces result to our neighbors list
    neighbors = []
    for i in range(k):
        neighbors.append(distances[i][0])
    return neighbors

k=10
avg_ratings=0
neighbors=get_neighbors(1,k)
for neighbor in neighbors:
    avg_ratings += movie_dict[neighbor][3]
    print(movie_dict[neighbor][0]+" "+str(movie_dict[neighbor][3]))
avg_ratings = avg_ratings/float(k)

Liar Liar (1997) size    485.000000
mean      3.156701
Name: 294, dtype: float64
Aladdin (1992) size    219.000000
mean      3.812785
Name: 95, dtype: float64
Willy Wonka and the Chocolate Factory (1971) size    326.000000
mean      3.631902
Name: 151, dtype: float64
Monty Python and the Holy Grail (1974) size    316.000000
mean      4.066456
Name: 168, dtype: float64
Full Monty, The (1997) size    315.000000
mean      3.926984
Name: 269, dtype: float64
George of the Jungle (1997) size    162.000000
mean      2.685185
Name: 259, dtype: float64
Beavis and Butt-head Do America (1996) size    156.000000
mean      2.788462
Name: 240, dtype: float64
Birdcage, The (1996) size    293.000000
mean      3.443686
Name: 25, dtype: float64
Home Alone (1990) size    137.000000
mean      3.087591
Name: 94, dtype: float64
Lion King, The (1994) size    220.000000
mean      3.781818
Name: 71, dtype: float64


In [23]:
print(avg_ratings)

size    262.900000
mean      3.438157
Name: 294, dtype: float64


In [24]:
movie_dict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 0.7749140893470791,
 size    452.000000
 mean      3.878319
 Name: 1, dtype: float64)

In [11]:
rating_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
user_id     100000 non-null int64
movie_id    100000 non-null int64
ratings     100000 non-null int64
dtypes: int64(3)
memory usage: 2.3 MB
